# Regresión: 1 neurona con 1 sola columna en X


## 1. Dataset con X, y

Supongamos un problema de **regresión** muy sencillo:

- **Entrada**: Años de experiencia ($ x $)  
- **Salida**: Salario ($ y $)

In [10]:
import numpy as np

X = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype=float)
y = np.array([1500, 1700, 2000, 2200, 2600, 3000, 3200, 3500, 4000, 4500], dtype=float)

## 2. Definir una neurona

Cuando solo hay una neurona, el forward pass se refiere a calcular la salida de esa neurona a partir de sus entradas, pesos, y función de activación.

Una neurona lineal simple (sin activación no lineal) se puede describir así:

$$
z = w \cdot x + b,
$$
$$
\hat{y} = z,
$$

donde:
- $x$ es tu dato de entrada (en este ejemplo, un solo valor: años de experiencia),
- $w$ es el peso,
- $b$ es el sesgo,
- $\hat{y}$ es la **predicción** de la neurona (el salario estimado).

> Si fuese una neurona con función de activación, por ejemplo $ y = f(z) $, lo mencionaríamos, pero para un problema de regresión, habitualmente se usa la **identidad** (o sea, no se pone activación adicional).

In [11]:
def forward(X, w, b):
    """
    Hace la predicción lineal para todos los puntos X.
    X: shape (N,) -> array de entradas
    w: escalar (peso)
    b: escalar (sesgo)
    Return: vector de predicciones (shape (N,))
    """
    # Z = w * X + b  (operación elemento a elemento)
    return w * X + b

## 3. Función de costo (MSE - Mean Squared Error)

Si tenemos $ N $ ejemplos $\{(x^{(i)}, y^{(i)})\}_{i=1}^N$, definimos la **función de costo** (o pérdida) como el **error cuadrático medio**:

$$
\text{MSE}(w, b) = \frac{1}{N} \sum_{i=1}^{N} \bigl(\hat{y}^{(i)} - y^{(i)}\bigr)^{2}
$$
donde $\hat{y}^{(i)} = w \cdot x^{(i)} + b$.

La función de costo mide la discrepancia entre las predicciones de la red $\hat{y}$ y las etiquetas verdaderas $y$.

**En código** (vectorizado con NumPy), si `X` es un array de tamaño $(N, )$ con todos los `x`, e `y` es un array de $(N, )$ con las etiquetas (salarios reales), podemos escribir:

In [12]:
import numpy as np

def mse_loss(y_pred, y_true):
    """
    y_pred: predicciones (N,)
    y_true: valores reales (N,)
    """
    return np.mean((y_pred - y_true)**2)

## 4. Derivadas: cómo ajustamos w y b

Una vez calculada la función de costo, queremos ajustar $\mathbf{W}$ y $\mathbf{b}$ para **minimizar** ese costo. 

Se utiliza el **Gradiente Descendente** y, para ello, necesitamos el **gradiente** de la función costo con respecto a cada parámetro $ w $ y $ b $.

Para **entrenar** la neurona queremos **minimizar** el MSE. 

Esto se hace a través de **descenso por gradiente**, que usa las **derivadas** del MSE respecto a $ w $ y $ b $.

**Una derivada** $\frac{\partial f}{\partial w}$ te dice cómo cambiará $ f $ si cambias un poco $ w $ por ejemplo. 

En **redes neuronales**, necesitamos las derivadas de la **función de costo** respecto a todos los parámetros (pesos w y sesgos b) para *saber en qué dirección* y *cuánto* ajustar esos parámetros de forma que minimicemos el error.

Usamos luego **descenso por gradiente** (o algoritmos más avanzados) para mover cada $ w, b $ en la dirección que *disminuye* el error.  

### Derivada de MSE respecto a $ w $

$$
\text{MSE}(w, b) = \frac{1}{N} \sum_{i=1}^{N} (w \cdot x^{(i)} + b - y^{(i)})^2.
$$
Tomando la derivada respecto a $ w $:
$$
\frac{\partial \text{MSE}}{\partial w} 
= \frac{2}{N} \sum_{i=1}^{N} \bigl(w \cdot x^{(i)} + b - y^{(i)}\bigr)\cdot x^{(i)}.
$$

> Observa que $\frac{\partial}{\partial w}(w \cdot x^{(i)}) = x^{(i)}$.

### Derivada de MSE respecto a $ b $

$$
\frac{\partial \text{MSE}}{\partial b} 
= \frac{2}{N} \sum_{i=1}^{N} \bigl(w \cdot x^{(i)} + b - y^{(i)}\bigr)\cdot 1.
$$

> Porque $\frac{\partial}{\partial b}(b) = 1$.

En la práctica, a menudo se ve una versión simplificada (sin el factor 2) si hemos definido MSE con $\frac12$.  
Pero aquí, como usamos `np.mean`, el factor 1/N ya está ahí, y el 2 se puede quedar o no, dependiendo de la convención. Lo importante es la consistencia.

### En código

En NumPy, podemos traducir estas sumas a operaciones de vectores:

In [13]:
def grad_w(X, y, w, b):
    """
    Calcula la derivada de MSE wrt w.
    X, y: shape (N,)
    w, b: escalares
    """
    N = len(X)
    # y_pred = w*X + b
    y_pred = forward(X, w, b)
    # (y_pred - y) -> shape (N,)
    # Multiplicamos elemento a elemento por X -> shape (N,)
    # Hacemos la media y multiplicamos por 2 (si seguimos la fórmula literal)
    return (2.0 / N) * np.sum((y_pred - y) * X)

def grad_b(X, y, w, b):
    """
    Calcula la derivada de MSE wrt b.
    """
    N = len(X)
    y_pred = forward(X, w, b)
    # (y_pred - y) -> shape (N,)
    return (2.0 / N) * np.sum(y_pred - y)

## 5. Entrenamiento (Descenso por gradiente)

La optimización en este contexto es el proceso de encontrar los valores de los parámetros $ w $ y $ b $ que minimizan la función de pérdida. Esto se logra iterativamente mediante el algoritmo de **descenso por gradiente**, que utiliza la siguiente regla:

$$
w_{\text{nuevo}} = w_{\text{actual}} - \eta \cdot \frac{\partial \text{Loss}}{\partial w}
$$
$$
b_{\text{nuevo}} = b_{\text{actual}} - \eta \cdot \frac{\partial \text{Loss}}{\partial b}
$$

- **$\eta$** es el *learning rate*.
- **$\frac{\partial \text{Loss}}{\partial w}$** y **$\frac{\partial \text{Loss}}{\partial b}$** son los gradientes que indican en qué dirección ajustar los parámetros para reducir la pérdida.
- Loss es la función costo, en este caso el MSE.

Por tanto para **ajustar** $ w $ y $ b $, se sigue un esquema iterativo:

1. **Inicializamos** $ w $ y $ b $ (por ejemplo, con valores aleatorios).
2. **Calculamos** las derivadas $\frac{\partial \text{MSE}}{\partial w}$ y $\frac{\partial \text{MSE}}{\partial b}$ con los datos completos.
3. **Actualizamos** $ w $ y $ b $ usando una tasa de aprendizaje $\eta$:
4. Repetimos durante un número de **epochs** (iteraciones) o hasta que el error sea suficientemente pequeño.

In [14]:
# 1. Inicializamos w y b en valores aleatorios pequeños
np.random.seed(42)
w = np.random.randn() * 0.01  # un valor aleatorio pequeño
b = np.random.randn() * 0.01

# 2. Configuración del entrenamiento
learning_rate = 0.01
num_epochs = 1000

# 3. Ciclo de entrenamiento
for epoch in range(num_epochs):
    
    # 1. Forward pass: calcular predicciones y pérdida
    y_pred = forward(X, w, b)
    loss_value = mse_loss(y_pred, y)
    
    # 2. Backward pass: calcular los gradientes
    dw = grad_w(X, y, w, b)
    db = grad_b(X, y, w, b)
    
    # 3. Actualizar parámetros weight y bias (descenso gradiente)
    # Optimización de w y b para minimizar la función costo
    w -= learning_rate * dw
    b -= learning_rate * db
    
    # (Opcional) Imprimir cada 50 iteraciones
    if (epoch+1) % 50 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss_value:.4f}, w: {w:.4f}, b: {b:.4f}")

print("\nEntrenamiento terminado.")
print(f"Parámetros finales: w = {w}, b = {b}")
y_pred_final = forward(X, w, b)
print("Predicciones finales:", y_pred_final)
print("Valores reales     :", y)

Epoch 50/1000, Loss: 140909.9149, w: 438.2242, b: 247.5753
Epoch 100/1000, Loss: 96616.3909, w: 417.1706, b: 394.1470
Epoch 150/1000, Loss: 67537.8563, w: 400.1120, b: 512.9060
Epoch 200/1000, Loss: 48447.9061, w: 386.2903, b: 609.1298
Epoch 250/1000, Loss: 35915.4244, w: 375.0914, b: 687.0947
Epoch 300/1000, Loss: 27687.8965, w: 366.0176, b: 750.2653
Epoch 350/1000, Loss: 22286.5550, w: 358.6655, b: 801.4489
Epoch 400/1000, Loss: 18740.5942, w: 352.7086, b: 842.9201
Epoch 450/1000, Loss: 16412.6841, w: 347.8820, b: 876.5220
Epoch 500/1000, Loss: 14884.4197, w: 343.9713, b: 903.7477
Epoch 550/1000, Loss: 13881.1198, w: 340.8026, b: 925.8072
Epoch 600/1000, Loss: 13222.4572, w: 338.2353, b: 943.6808
Epoch 650/1000, Loss: 12790.0476, w: 336.1551, b: 958.1628
Epoch 700/1000, Loss: 12506.1723, w: 334.4696, b: 969.8967
Epoch 750/1000, Loss: 12319.8092, w: 333.1039, b: 979.4041
Epoch 800/1000, Loss: 12197.4625, w: 331.9974, b: 987.1074
Epoch 850/1000, Loss: 12117.1424, w: 331.1009, b: 993.34

In [15]:
mse_loss(y_pred_final, y)

np.float64(12006.705564050028)

In [16]:
def mae_loss(y_pred, y_true):
    return np.mean(np.abs(y_pred - y_true))

mae_loss(y_pred_final, y)

np.float64(88.3189466910253)

In [17]:
def r2_score(y_pred, y_true):
    # Suma total de cuadrados (variación de los datos respecto a la media)
    total_variance = np.sum((y_true - np.mean(y_true)) ** 2)
    # Suma de los residuos al cuadrado (errores del modelo)
    residual_variance = np.sum((y_true - y_pred) ** 2)
    # R^2: 1 - (varianza explicada por el error / varianza total)
    return 1 - (residual_variance / total_variance)

r2_score(y_pred_final, y)

np.float64(0.9865936740017307)

## ¿Cómo se extendería a red con más neuronas, capas y entradas X?

1. **Más entradas**: Si $ x $ fuese un vector ($ x_1, x_2, \dots, x_n $), la neurona sería
   $$
   z = w_1 x_1 + w_2 x_2 + \dots + w_n x_n + b.
   $$
   Y en NumPy usaríamos `np.dot(w, X) + b`.
   
2. **Función de activación**: Si añades $ y = f(z) $ (sigmoide, ReLU, etc.), la derivada se multiplica por $ f'(z) $. Esto es la **regla de la cadena**.

3. **Varias neuronas** (capas): Se compone cada salida como entrada de la siguiente capa. Para derivar, se aplica la regla de la cadena para cada capa. En NumPy se vectoriza para que todos los ejemplos se procesen en bloque (batch).

Pero **los fundamentos** son estos: el cálculo de derivadas (slopes) de la función de costo respecto a tus parámetros es lo que te permite corregirlos y mejorar tu modelo.  